In [ ]:
import os
os.chdir(os.path.dirname(os.path.realpath(__file__)))

import torch
from preprocessing import TextPreprocessor
import config as cfg
import helpers
import models
import os
import data
from torch.utils.data import DataLoader
from eval import Evaluation
from train import get_comet_api_key  # TODO refactor
import uuid
from lime import lime_text

config = cfg.default_config

train_df = helpers.get_combined_dataframes(helpers.get_datasets_paths(config, 'train'))

text_proc = TextPreprocessor(config, train_df)

model = models.get_model(text_proc.n_tokens, config)
model.to(config['device'])
checkpoint = torch.load(os.path.join('checkpoints', config['load_checkpoint']), map_location=torch.device(config['device']))
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

In [ ]:
def batch_predict(texts):
        tokens_batch, _ = text_proc.get_model_input(texts)
        predicated_labels, probs = model(tokens_batch.to(config['device']), return_probs=True)
        return probs.detach().cpu().numpy()

In [ ]:
explainer = lime_text.LimeTextExplainer(class_names=['PAL', 'LEB', 'JOR', 'SYR']

In [ ]:
test_inputs = 'وهى حماتك بتكشر من الطريق الا يكون فى اشى'
explanation = explainer.explain_instance(test_inputs, batch_predict, labels=[0])
explanation.show_in_notebook(text=True)